In [3]:
from itertools import chain
from pathlib import Path

from IPython.display import Markdown, display
from tqdm.notebook import tqdm

import hashlib
from os import environ
import pyperclip
import subprocess

from githelpers import open_pbcopy
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [4]:
cached = ghapicache.GhApiCache()
ghapi = cached.api

In [6]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

# List active members

This is to be used for crediting in presentations, etc.

In [7]:
text = []
for team, title in [
    ('code-lead', 'Code lead'),
    ('core', 'Core members'),
    ('core-advisor', 'Core advisors'),
    ('affiliates', 'Affiliates'),]:
    text.append(f"## {title}")
    m = [user_cache[username] for username in TEAMS[team].members]
    for member in sorted(m, key=release_notes.get_last_name):
        text.append("- " + release_notes.format_user(member))
    text.append("")

display(Markdown("\n".join(text)))

## Code lead
- Seth R. Johnson *(ORNL)*

## Core members
- Elliott Biondo *(ORNL)*
- Julien Esseiva *(LBNL)*
- Hayden Hollenbeck *(UVA)*
- Seth R. Johnson *(ORNL)*
- Soon Yung Jun *(FNAL)*
- Guilherme Lima *(FNAL)*
- Amanda Lund *(ANL)*
- Ben Morgan *(U Warwick)*
- Sakib Rahman *(BNL)*
- Stefano Tognini *(ORNL)*

## Core advisors
- Philippe Canal *(FNAL)*
- Marcel Demarteau *(ORNL)*
- Tom Evans *(ORNL)*

## Affiliates
- Lance Bullerwell *(ORNL)*
- Wouter Deconinck *(U Manitoba)*
- Sam Eriksen *(U Bristol)*
- Steven Hamilton *(ORNL)*
- Paul Romano *(ANL)*
- Frederic Suter *(ORNL)*
- Sandro Wenzel *(CERN)*


## Release note generation

- Merge base should be all commits *already* released (skip documenting)
- Target branch is the one where the release candidate is
- Previous major branch allows all "v.x" contributors to be credited

In [8]:
ReleaseMetadata = release_notes.ReleaseMetadata

# All commits from all time
all_md = ReleaseMetadata(
    merge_bases=['v0.0.0'],
)

all_prs = release_notes.PullRequestRange(all_md)
count_contrib = release_notes.ContributionCounter(cached)
for pr in tqdm(all_prs.pull_ids):
    count_contrib(pr)
authors = count_contrib.sorted().author
authors

Can't match log subject to PR: Format code base (clang-format version 11.0.1)


  0%|          | 0/1414 [00:00<?, ?it/s]

{'sethrj': 840,
 'amandalund': 205,
 'esseivaju': 88,
 'stognini': 48,
 'whokion': 46,
 'pcanal': 45,
 'elliottbiondo': 33,
 'mrguilima': 31,
 'drbenmorgan': 22,
 'paulromano': 15,
 'hhollenb': 13,
 'vrpascuzzi': 7,
 'tmdelellis': 6,
 'VHLM2001': 4,
 'dalg24': 3,
 'lebuller': 3,
 'DoaaDeeb': 1,
 'aprokop': 1,
 'hartsw': 1,
 'ptheywood': 1,
 'rahmans1': 1}

In [9]:
print("Missing ORCIDs:")
print(" ".join(f"@{u}" for u in authors if user_cache[u].orcid is None))

Missing ORCIDs:
@VHLM2001 @DoaaDeeb


## Release

In [10]:
# Major release
major_md = ReleaseMetadata(
    release='0.6.0',
    merge_bases=['v0.5.0', 'v0.5.2'],
)

# Minor release
minor_md = ReleaseMetadata(
    release='0.5.3',
    merge_bases=['v0.5.2'],
    target_branch='backports/v0.5'
)

In [11]:
if 1:
    # Backport release:
    release_md = minor_md
    note_body =  """
Version {release} is a minor update to Celeritas featuring an important physics bugfix
to Urban MSC and additional minor fixes targeting CUDA VecGeom compatibility.
"""
else:
    # Major release
    release_md = major_md
    note_body = """
Version {release} is a major update to Celeritas featuring:

- 

A few minor features are noteworthy:

- 

Important changes:

- 

Notable bug fixes include:

- 

Some interfaces have been removed:

- 
"""

In [12]:

prs = release_notes.PullRequestRange(release_md)
sorted_pulls = release_notes.SortedPulls(cached)
count_contrib = release_notes.ContributionCounter(cached)
for pr_id in tqdm(prs.pull_ids):
    try:
        count_contrib(pr_id)
        sorted_pulls.add(pr_id)
    except Exception as e:
        print(f"Error adding PR #{pr_id}: {e}")

reviewers = count_contrib.sorted().reviewer
for login in tqdm(reviewers):
    user_cache[login]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
def make_notes(cls, release_md, note_body):
    prev = release_md.merge_bases[0]
    other = release_md.merge_bases[1:]
    if not other:
        change_str = f"Changes since {prev} follow."
    else:
        assert len(other) == 1
        change_str = f"Changes since {prev}, excluding those released in {other[0]}, follow."

    notes = cls(release_md, note_body)
    notes.paragraph(change_str)
    notes.sorted_pulls(sorted_pulls)
    notes.reviewers(reviewers, user_cache)
    notes.changelog_line("celeritas-project", "celeritas")

    return notes

In [ ]:
rst_notes = make_notes(release_notes.RstNotes, release_md, note_body)

with open_pbcopy() as pb:
    rst_notes.write(pb)

# Draft github release

In [30]:
ghapi = cached.api

In [ ]:
markdown_notes = make_notes(release_notes.MarkdownNotes, release_md, note_body)


In [43]:
release = release_notes.find_release(ghapi, release_md.release)
if not release:
    release_md = release_notes.create_release(ghapi, release_md, str(markdown_notes))

No release found for version: 0.5.3
Draft release 0.5.3 created: https://github.com/celeritas-project/celeritas/releases/tag/untagged-3dbf57ae5e31f3508768


In [45]:
artifact_url, artifact_tgz = release_notes.get_or_upload_tarball(cached, release)
print(f"Artifact URL: {artifact_url}")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#assert 0
sha256_hash = hashlib.sha256(artifact_tgz).hexdigest()
pyperclip.copy(f'version("{release_md.release}", sha256="{sha256_hash}")')
print("Spack version copied to clipboard!")
subprocess.check_call(
    [
        "open",
        Path(environ["SPACK_ROOT"])
        / "var/spack/repos/builtin/packages/celeritas/package.py",
    ]
)

## TODO: push to zenodo